# Importações

In [16]:
import xml.etree.ElementTree as xet
from glob import glob
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import precision_score, recall_score
from tensorflow.python.client import device_lib

In [17]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10269862178430160486
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7604932448
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5111270545293332214
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


2024-04-08 13:49:12.963472: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-08 13:49:12.989923: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-04-08 13:49:12.993249: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-04-08 13:49:13.069060: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-08 13:49:13.069307: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 na

# Tratando Dataset

In [2]:
!curl -L -o /tmp/tcc_alpd/archive.tar.gz "https://www.dropbox.com/scl/fi/khnnv0x8pcpbzlgfevqac/archive.tar.gz?rlkey=55w3pkxp02szjpqw22df7hmn8&dl=0"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   256    0   256    0     0    196      0 --:--:--  0:00:01 --:--:--   196
100   491    0   491    0     0    206      0 --:--:--  0:00:02 --:--:--   536
100  202M  100  202M    0     0  8840k      0  0:00:23  0:00:23 --:--:--  9.8M


In [3]:
!tar xfv /tmp/tcc_alpd/archive.tar.gz -C /tmp/tcc_alpd

archive/
archive/.DS_Store
archive/images/
archive/annotations/
archive/annotations/Cars213.xml
archive/annotations/Cars207.xml
archive/annotations/Cars398.xml
archive/annotations/Cars367.xml
archive/annotations/Cars401.xml
archive/annotations/Cars415.xml
archive/annotations/Cars373.xml
archive/annotations/Cars429.xml
archive/annotations/Cars47.xml
archive/annotations/Cars53.xml
archive/annotations/Cars165.xml
archive/annotations/Cars171.xml
archive/annotations/Cars159.xml
archive/annotations/Cars84.xml
archive/annotations/Cars90.xml
archive/annotations/Cars91.xml
archive/annotations/Cars85.xml
archive/annotations/Cars158.xml
archive/annotations/Cars170.xml
archive/annotations/Cars164.xml
archive/annotations/Cars52.xml
archive/annotations/Cars46.xml
archive/annotations/Cars428.xml
archive/annotations/Cars414.xml
archive/annotations/Cars372.xml
archive/annotations/Cars366.xml
archive/annotations/Cars400.xml
archive/annotations/Cars399.xml
archive/annotations/Cars206.xml
archive/annotati

In [6]:
def generate_dataframe_from_xml_annotations(directory):
    path = glob(directory)
    labels_dict = dict(filepath=[], xmin=[], xmax=[], ymin=[], ymax=[])
    
    for filename in path:
        info = xet.parse(filename)
        root = info.getroot()
        member_object = root.find('object')
        labels_info = member_object.find('bndbox')
        xmin = int(labels_info.find('xmin').text)
        xmax = int(labels_info.find('xmax').text)
        ymin = int(labels_info.find('ymin').text)
        ymax = int(labels_info.find('ymax').text)

        labels_dict['filepath'].append(filename)
        labels_dict['xmin'].append(xmin)
        labels_dict['xmax'].append(xmax)
        labels_dict['ymin'].append(ymin)
        labels_dict['ymax'].append(ymax)

    df = pd.DataFrame(labels_dict)
    return df

def getFilename(filepath):
    filename_image = xet.parse(filepath).getroot().find('filename').text
    filepath_image = os.path.join('/tmp/tcc_alpd/archive/images', filename_image)
    return filepath_image

def get_image_paths_from_dataframe(df):
    return df['filepath'].apply(getFilename)

In [8]:
df = generate_dataframe_from_xml_annotations('/tmp/tcc_alpd/archive/annotations/*.xml')
image_path = get_image_paths_from_dataframe(df)
df

,filepath,xmin,xmax,ymin,ymax
0,/tmp/tcc_alpd/archive/annotations/Cars400.xml,303,340,191,217
1,/tmp/tcc_alpd/archive/annotations/Cars104.xml,195,244,266,282
2,/tmp/tcc_alpd/archive/annotations/Cars224.xml,181,243,250,275
3,/tmp/tcc_alpd/archive/annotations/Cars311.xml,295,355,166,185
4,/tmp/tcc_alpd/archive/annotations/Cars319.xml,167,232,228,259
...,...,...,...,...,...
428,/tmp/tcc_alpd/archive/annotations/Cars396.xml,181,269,264,289
429,/tmp/tcc_alpd/archive/annotations/Cars388.xml,149,255,130,158
430,/tmp/tcc_alpd/archive/annotations/Cars92.xml,281,393,185,271
431,/tmp/tcc_alpd/archive/annotations/Cars126.xml,103,293,99,201


In [11]:
def preprocess_data(image_paths, labels):
    processed_data = []
    processed_output = []

    for idx, img_path in enumerate(image_paths):
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))  # Redimensiona a imagem para 224x224
        img = img / 255.0  # Normaliza os valores dos pixels para o intervalo [0, 1]

        height, width, _ = img.shape
        xmin, xmax, ymin, ymax = labels[idx]
        nxmin, nxmax = xmin / width, xmax / width
        nymin, nymax = ymin / height, ymax / height
        label_norm = (nxmin, nxmax, nymin, nymax)

        processed_data.append(img)
        processed_output.append(label_norm)

    return processed_data, processed_output

processed_data, processed_output = preprocess_data(image_path, df.iloc[:, 1:].values)

In [15]:
def split_train_test_data(processed_data, processed_output, train_size=0.8, random_state=None):
    X = np.array(processed_data, dtype=np.float32)
    y = np.array(processed_output, dtype=np.float32)

    x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=random_state)

    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = split_train_test_data(processed_data, processed_output, train_size=0.8, random_state=0)
print("Tamanhos dos conjuntos de treino e teste:")
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

Tamanhos dos conjuntos de treino e teste:
x_train shape: (346, 224, 224, 3)
x_test shape: (87, 224, 224, 3)
y_train shape: (346, 4)
y_test shape: (87, 4)


# Treinando o Modelo

In [18]:
def create_custom_model(input_shape=(224, 224, 3), num_classes=4):
    base_model = InceptionResNetV2(weights="imagenet", include_top=False, input_shape=input_shape)

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    x = Dense(500, activation="relu")(x)
    x = Dense(250, activation="relu")(x)
    outputs = Dense(num_classes, activation='sigmoid')(x)

    custom_model = Model(inputs=base_model.input, outputs=outputs)
    
    return custom_model

model = create_custom_model()

model.summary()

2024-04-08 13:55:58.980023: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-04-08 13:55:58.980559: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-08 13:55:58.981150: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.71GHz coreCount: 46 deviceMemorySize: 7.78GiB deviceMemoryBandwidth: 417.23GiB/s
2024-04-08 13:55:58.981261: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2024-04-08 13:55:58.981353: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2024-04-08 13:55:58.981429: I tensorflow/stream_executor/platform/

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [19]:
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))

In [70]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=50,
                    validation_data=(x_test,y_test))

Epoch 1/50
35/35 [==============================] - 17s 207ms/step - loss: 0.0499 - val_loss: 0.0305
Epoch 2/50
35/35 [==============================] - 5s 139ms/step - loss: 0.0185 - val_loss: 0.0223
Epoch 3/50
35/35 [==============================] - 5s 139ms/step - loss: 0.0092 - val_loss: 0.0173
Epoch 4/50
35/35 [==============================] - 5s 139ms/step - loss: 0.0071 - val_loss: 0.0124
Epoch 5/50
35/35 [==============================] - 5s 140ms/step - loss: 0.0042 - val_loss: 0.0118
Epoch 6/50
35/35 [==============================] - 5s 140ms/step - loss: 0.0034 - val_loss: 0.0098
Epoch 7/50
35/35 [==============================] - 5s 140ms/step - loss: 0.0026 - val_loss: 0.0108
Epoch 8/50
35/35 [==============================] - 5s 140ms/step - loss: 0.0018 - val_loss: 0.0117
Epoch 9/50
35/35 [==============================] - 5s 140ms/step - loss: 0.0030 - val_loss: 0.0118
Epoch 10/50
35/35 [==============================] - 5s 140ms/step - loss: 0.0019 - val_loss: 0.008

In [71]:
model.save('./license_plate_detection.h5')

In [72]:
y_pred = model.predict(x_test)

In [73]:
y_test_flat = np.argmax(y_test, axis=1)
y_pred_flat = np.argmax(y_pred, axis=1)

precision = precision_score(y_test_flat, y_pred_flat, average='macro')
recall = recall_score(y_test_flat, y_pred_flat, average='macro')

print("Precision:", precision)
print("Recall:", recall)

Precision: 0.8700322234156821
Recall: 0.8724324324324324
